In [25]:
import numpy as np
import cv2 
import scipy
import matplotlib.pylab as plt
from skimage import io

In [26]:
vid_left = cv2.VideoCapture('./Dataset-1/left_output-1.avi') 
template_1_url = "https://raw.githubusercontent.com/jbinteam/010723305/main/images/final_exam/Templates/Template-1.png"

template_1_img = io.imread(template_1_url)
template_1_gray = cv2.cvtColor(template_1_img, cv2.COLOR_BGR2GRAY) 


In [27]:
def feature_object_detection(template_1_img, template_1_gray, query_img, query_gray, MIN_MATCH_NUMBER) :
    template_1_kpts, template_1_desc = sift.detectAndCompute(template_1_gray, None)
    query_1_kpts, query_1_desc = sift.detectAndCompute(query_1_gray, None)
    matches = bf.knnMatch(template_1_desc, query_1_desc, k=2)
    good_matches = list()
    good_matches_list = list()
    for m, n in matches :
        if m.distance < 0.73*n.distance :
            good_matches.append(m)
            good_matches_list.append([m])

    if len(good_matches) > MIN_MATCH_NUMBER : 
        src_pts = np.float32([ template_1_kpts[m.queryIdx].pt for m in good_matches ]).reshape(-1,1,2)
        dst_pts = np.float32([ query_1_kpts[m.trainIdx].pt for m in good_matches ]).reshape(-1,1,2)

        H, inlier_masks = cv2.findHomography(src_pts, dst_pts, cv2.RANSAC, 1.5) # H RANSAC 
        #get the bounding box around template image
        h, w = template_1_img.shape[:2]
        template_box = np.float32([[0, 0], [0, h-1], [w-1, h-1], [w-1, 0]]).reshape(-1,1,2) 
        transformed_box = cv2.perspectiveTransform(template_box, H)

        detected_img = cv2.polylines( frame, [np.int32(transformed_box)], True, (0,255,0), 1, cv2.LINE_AA)
        drawmatch_img = cv2.drawMatchesKnn(template_1_img, template_1_kpts, detected_img, query_1_kpts, good_matches_list, None, flags=2, matchesMask=inlier_masks)

        return detected_img, drawmatch_img
    else :
        print('Keypoints not enough')
        return

In [28]:
if (vid_left.isOpened()== False): 
  print("Error opening video stream or file")
      
while (vid_left.isOpened()) : 
    ret,  frame = vid_left.read()  

    if (ret ==True) :
        
        kernel = np.array([[0,0,0], [0,1.7,0],[0,0,0]])  
        
        query_1_img = cv2.filter2D(src= frame, ddepth= -1, kernel= kernel) 
        query_1_gray = cv2.cvtColor(query_1_img, cv2.COLOR_BGR2GRAY) 
        
        sift = cv2.SIFT_create() 
        bf = cv2.BFMatcher()
        MIN_MATCH_NUMBER = 10
        detected_img, drawmatch_img = feature_object_detection(template_1_img, template_1_gray, query_1_img, query_1_gray, MIN_MATCH_NUMBER)
            
        cv2.imshow('detected',detected_img)

        if cv2.waitKey(25) & 0xFF == ord('q'):
          break
     
    else :
        break
    
vid_left.release  ()
cv2.destroyAllWindows ()